### Rijksmuseum API

In [2]:
import requests

SEARCH_URL = "https://data.rijksmuseum.nl/search/collection"

def search_portraits(title=None, creator=None, t='painting'):
    params = {
        "creator": creator, 
        "title": title,
        # "imageAvailable": "true",
        "type": t
    }

    r = requests.get(SEARCH_URL, params=params)
    r.raise_for_status()
    data = r.json()

    return data

In [115]:
def parse_artwork_details(data: dict) -> dict:
    """
    Extracts useful structured fields from Rijksmuseum Linked.Art objects
    """
    
    en_code = "http://vocab.getty.edu/aat/300388277"
    nl_code = "http://vocab.getty.edu/aat/300388256" #dutch language has more information

    unit_map = {
        "http://vocab.getty.edu/aat/300379098": "cm",
        "http://vocab.getty.edu/aat/300379226": "kg",
    }
    
    attr_map = {
        "https://id.rijksmuseum.nl/22011": "hoogte",
        "https://id.rijksmuseum.nl/22012": "breedte",
        "https://id.rijksmuseum.nl/220217": "gewicht",
    }

    # ------------ TITLE ------------
    # First look for Dutch version
    
    title = None
    for s in data.get("subject_of", []):
        for part in s.get("part", []):
            for sub in part.get("part", []):
                if sub.get("type") == "Name":
                    langs = sub.get("language", [])
                    if any(l.get("id") == nl_code for l in langs):
                        title = sub.get("content")
                        break
            if title:
                break
        if title:
            break

    # fallback: take any title if no English was found
    if not title:
        for s in data.get("subject_of", []):
            for part in s.get("part", []):
                for sub in part.get("part", []):
                    if sub.get("type") == "Name":
                        title = sub.get("content")
                        break
                if title:
                    break
            if title:
                break

    # ------------ ARTIST / MAKER ------------
    artist_name = None
    artist_id = None
    
    prod = data.get("produced_by")
    if isinstance(prod, dict):
        for part in prod.get("part", []):
            # get the person URI
            for agent in part.get("carried_out_by", []):
                artist_id = agent.get("id")
    
            # read Dutch referred_to_by labels
            for ref in part.get("referred_to_by", []):
                if ref.get("type") == "LinguisticObject":
                    langs = ref.get("language", [])
                    if any(l.get("id") == nl_code for l in langs):
                        artist_name = ref.get("content")
                        break
    
            # fallback: any referred_to_by without language filter
            if artist_name is None:
                for ref in part.get("referred_to_by", []):
                    if ref.get("type") == "LinguisticObject":
                        artist_name = ref.get("content")
                        break

    # ------------ YEAR ------------
    year = None
    ts = prod.get("timespan") if prod else None
    if isinstance(ts, dict):
        # Try identified_by textual year first
        if isinstance(ts.get("identified_by"), list):
            for ident in ts["identified_by"]:
                c = ident.get("content")
                if c and any(ch.isdigit() for ch in c):
                    year = c
                    break

        # fallback to machine timestamps
        if year is None:
            b = ts.get("begin_of_the_begin")
            if b: 
                year = b[:4]

    # ------------ DESCRIPTION ------------
    descriptions_nl = []

    for entry in data.get("subject_of", []):
        langs = entry.get("language", [])
        if not any(l.get("id") == nl_code for l in langs):
            continue
    
        # level 1: direct content
        if "content" in entry:
            descriptions_nl.append(entry["content"])
    
        # level 2: parts
        for p in entry.get("part", []):
            if "content" in p:
                descriptions_nl.append(p["content"])
            for sub in p.get("part", []):
                if "content" in sub:
                    descriptions_nl.append(sub["content"])
    # deduplicate
    descriptions_nl = list(dict.fromkeys(descriptions_nl))
    
    # ------------ LOCATION ------------
    location = None
    room = None
    loc = data.get('current_location', [])

    if loc:
        for item in loc.get("identified_by", []):
        
            # 1. Extract identifier
            if item.get("type") == "Identifier":
                if "content" in item:
                    room = item["content"]
        
            # 2. Extract location name in english
            if item.get("type") == "Name":
                langs = item.get("language", [])
                if any(l.get("id") == nl_code for l in langs):
                    parts = item.get("part", [])
                    names = [p.get("content") for p in parts if p.get("content")]
                    location = " ".join(names)
                    
    # ------------ DIMENSION ------------        
    entries = []
    
    for item in data.get("dimension", []):
        if item.get("type") != "Dimension":
            continue
        
        value = item.get("value")
        unit_id = item.get("unit", {}).get("id")
        unit = unit_map.get(unit_id, "")
        
        # get attribute from classified_as
        attr = None
        for c in item.get("classified_as", []):
            a = attr_map.get(c.get("id"))
            if a:
                attr = a
        
        # gather the Dutch annotation text
        annotation = None
        for ref in item.get("referred_to_by", []):
            langs = ref.get("language", [])
            if any(l.get("id") == nl_code for l in langs):
                annotation = ref.get("content")
    
        if attr and value and unit:
            entries.append(f"{attr} {value} {unit}" + (f" ({annotation})" if annotation else ""))
            
    dimension_str = " x ".join(entries)

    # ------------ MATERIAL ------------        

    material_code = "http://vocab.getty.edu/aat/300435429"
    
    materials = []
    
    for item in data.get("referred_to_by", []):
        if item.get("type") != "LinguisticObject":
            continue
        
        langs = item.get("language", [])
        if not any(l.get("id") == nl_code for l in langs):
            continue
    
        classes = item.get("classified_as", [])
        if not any(c.get("id") == material_code for c in classes):
            continue
    
        content = item.get("content")
        if content:
            materials.append(content)
    
    materials = list(dict.fromkeys(materials))

    
    return {
        "title": title,
        "artist": artist_name,
        "year": year,
        "descriptions": descriptions_nl,
        "location": location,
        "room": room,
        "dimension": dimension_str,
        "material": materials,
        "source": data.get("id"),
    }

In [126]:
# what to search
creator = 'Van Gogh'
title = 'self portrait'

data = search_portraits(title=title, creator=creator)
artwork_id = data["orderedItems"][0]['id']

extracted_info = requests.get(
    artwork_id,
    headers={"Accept": "application/ld+json"}
).json()

# print(extracted_info.keys())
extracted_data = parse_artwork_details(extracted_info)


In [127]:
extracted_data  # what we extract

{'title': 'Zelfportret',
 'artist': 'Vincent van Gogh',
 'year': '1887',
 'descriptions': ['Vincent van Gogh (1853–1890), olieverf op karton, 1887',
  'Zelfportret',
  'Nadat hij van zijn broer Theo had gehoord over de nieuwe kleurige Franse schilderkunst ging Vincent in 1886 in Parijs wonen. Al snel probeerde hij de Franse stijl uit op een aantal zelfportretten. Dit deed hij vooral om de kosten voor een model uit te sparen. Hier schilderde hij zichzelf als een mondain geklede Parijzenaar, met losse, ritmische penseelstreken in opvallende kleuren.'],
 'location': '19de Eeuw Hoofdgebouw Haagse School / Amsterdamse impressionisten / Van Gogh en tijdgenoten',
 'room': 'HG-1.18',
 'dimension': 'breedte 34 cm x hoogte 42 cm',
 'material': ['olieverf op karton'],
 'source': 'https://id.rijksmuseum.nl/200109794'}